<a href="https://colab.research.google.com/github/vasudevgupta7/bigbird/blob/main/notebooks/evaluate-flax-natural-questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
!git clone https://github.com/vasudevgupta7/bigbird
!cd bigbird && pip3 install -qr requirements.txt

!pip3 install -q flax

In [ ]:
!mkdir natural-questions-validation
!wget https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/natural_questions-validation.arrow -P natural-questions-validation
!wget https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/dataset_info.json -P natural-questions-validation
!wget https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/state.json -P natural-questions-validation

In [11]:
from datasets import load_from_disk

dataset = load_from_disk("natural-questions-validation")
dataset

Dataset({
    features: ['id', 'document', 'question', 'annotations'],
    num_rows: 7830
})

In [12]:
def format_dataset(sample):
  question = sample['question']['text']
  context = sample['document']['tokens']['token']
  is_html = sample['document']['tokens']['is_html']
  long_answers = sample['annotations']['long_answer']
  short_answers = sample['annotations']['short_answers']

  context_string =  " ".join([context[i] for i in range(len(context)) if not is_html[i]])

  # 0 - No ; 1 - Yes
  for answer in sample['annotations']['yes_no_answer']:
    if answer == 0 or answer == 1:
      return {"question": question, "context": context_string, "short": [], "long": [], "category": "no" if answer == 0 else "yes"}

  short_targets = []
  for s in short_answers:
    short_targets.extend(s['text'])
  short_targets = list(set(short_targets))

  long_targets = []
  for s in long_answers:
    if s['start_token'] == -1:
      continue
    answer = context[s['start_token']: s['end_token']]
    html = is_html[s['start_token']: s['end_token']]
    new_answer = " ".join([answer[i] for i in range(len(answer)) if not html[i]])
    if new_answer not in long_targets:
      long_targets.append(new_answer)

  category = "long_short" if len(short_targets + long_targets) > 0 else "null"

  return {"question": question, "context": context_string, "short": short_targets, "long": long_targets, "category": category}

In [13]:
dataset = dataset.map(format_dataset).remove_columns(["annotations", "document", "id"])

In [14]:
short_validation_dataset = dataset.filter(lambda x: (len(x['question']) + len(x['context'])) < 4 * 4096)
short_validation_dataset = short_validation_dataset.filter(lambda x: x['category'] != "null")
short_validation_dataset

Dataset({
    features: ['category', 'context', 'long', 'question', 'short'],
    num_rows: 1823
})

In [15]:
PUNCTUATION_SET_TO_EXCLUDE = set(''.join(['‘', '’', '´', '`', '.', ',', '-', '"']))

def get_sub_answers(answers, begin=0, end=None):
  return [" ".join(x.split(" ")[begin:end]) for x in answers if len(x.split(" ")) > 1]

def expand_to_aliases(given_answers, make_sub_answers=False):
  if make_sub_answers:
    # if answers are longer than one word, make sure a predictions is correct if it coresponds to the complete 1: or :-1 sub word
    # *e.g.* if the correct answer contains a prefix such as "the", or "a"
    given_answers = given_answers + get_sub_answers(given_answers, begin=1) + get_sub_answers(given_answers, end=-1)
  answers = []
  for answer in given_answers:
    alias = answer.replace('_', ' ').lower()
    alias = ''.join(c if c not in PUNCTUATION_SET_TO_EXCLUDE else ' ' for c in alias)
    answers.append(' '.join(alias.split()).strip())
  return set(answers)

In [16]:
def get_best_valid_start_end_idx(start_scores, end_scores, top_k=1, max_size=100):
    best_start_scores, best_start_idx = jax.lax.top_k(start_scores, top_k)
    best_end_scores, best_end_idx = jax.lax.top_k(end_scores, top_k)

    widths = best_end_idx[:, None] - best_start_idx[None, :]
    mask = jnp.logical_or(widths < 0, widths > max_size)
    scores = (best_end_scores[:, None] + best_start_scores[None, :]) - (1e8 * mask)
    best_score = jnp.argmax(scores).item()

    return best_start_idx[best_score % top_k], best_end_idx[best_score // top_k]

In [40]:
import jax
import jax.numpy as jnp

@jax.jit
def forward(*args, **kwargs):
  start_logits, end_logits, pooled_logits = model(*args, **kwargs)
  return start_logits, end_logits, jnp.argmax(pooled_logits, axis=-1)

def evaluate(example):
    # encode question and context so that they are seperated by a tokenizer.sep_token and cut at max_length
    inputs = tokenizer(example["question"], example["context"], return_tensors="jax", max_length=4096, padding="max_length", truncation=True)

    start_scores, end_scores, category = forward(**inputs)

    predicted_category = CATEGORY_MAPPING[category.item()]

    example['targets'] = example['long'] + example['short']
    if example['category'] in ['yes', 'no', 'null']:
      example['targets'] = [example['category']]
    example['has_tgt'] = example['category'] != 'null'
    # Now target can be: "yes", "no", "null", "list of long & short answers"

    if predicted_category in ['yes', 'no', 'null']:
        example['output'] = [predicted_category]
        example['match'] = example['output'] == example['targets']
        example['has_pred'] = predicted_category != 'null'
        return example

    max_size = 38 if predicted_category == "short" else 1024
    start_score, end_score = get_best_valid_start_end_idx(start_scores[0], end_scores[0], top_k=8, max_size=max_size)

    input_ids = inputs["input_ids"][0].tolist()
    example["output"] = [tokenizer.decode(input_ids[start_score: end_score+1])]

    answers = expand_to_aliases(example["targets"], make_sub_answers=True)
    predictions = expand_to_aliases(example["output"])

    # some preprocessing to both prediction and answer
    answers = set(["".join(a.split()) for a in answers])
    predictions = set(["".join(p.split()) for p in predictions])
    predictions = set([s for s in predictions if s not in ["``", "''", "`", "'"]])

    # if there is a common element, it's a exact match
    example["match"] = len(list(answers & predictions)) > 0
    example["has_pred"] = predicted_category != 'null' and len(predictions) > 0

    return example

In [18]:
import numpy as np
from train_nq_flax import FlaxBigBirdForNaturalQuestions
from params import CATEGORY_MAPPING
from transformers import BigBirdTokenizerFast

CATEGORY_MAPPING = {v: k for k, v in CATEGORY_MAPPING.items()}
CATEGORY_MAPPING

{0: 'null', 1: 'short', 2: 'long', 3: 'yes', 4: 'no'}

In [19]:
MODEL_ID = "vasudevgupta/flax-bigbird-natural-questions"

model = FlaxBigBirdForNaturalQuestions.from_pretrained(MODEL_ID)
tokenizer = BigBirdTokenizerFast.from_pretrained(MODEL_ID)

In [41]:
def evaluate_print(example, verbose=False):
  example = evaluate(example)
  if verbose:
    print("TARGET", example["short"] if len(example["short"]) > 0 else example['long'])
    print("PREDICTION", example["output"], end="\n\n")
  return example

short_validation_dataset = short_validation_dataset.map(evaluate_print)

In [42]:
total = len(short_validation_dataset)
matched = len(short_validation_dataset.filter(lambda x: x["match"] == 1))
print("EM score:", (matched / total)*100, "%")


EM score: 55.12890839275919 %


We are getting **Exact Match ~ 55.12** 💥💥
<!-- # this f1 is as per official nq script from here (https://github.com/google-research-datasets/natural-questions/blob/master/nq_eval.py)-->
<!-- 
# normal: 50.5
# no space: 55.12 -->